In [3]:
!pip install hanja

     -------------------------------------- 120.2/120.2 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 265.0/265.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 188.5/188.5 kB 5.8 MB/s eta 0:00:00
     ------------------------------------- 204.5/204.5 kB 12.9 MB/s eta 0:00:00
     ------------------------------------- 204.4/204.4 kB 12.1 MB/s eta 0:00:00
     -------------------------------------- 204.1/204.1 kB 6.1 MB/s eta 0:00:00
     -------------------------------------- 204.1/204.1 kB 6.1 MB/s eta 0:00:00
     ------------------------------------- 203.6/203.6 kB 12.9 MB/s eta 0:00:00
     -------------------------------------- 203.6/203.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.7.0 requires pyyaml>=5.3.1, but you have pyyaml 5.1.2 which is incompatible.
anaconda-client 1.11.0 requires urllib3>=1.26.4, but you have urllib3 1.25.11 which is incompatible.

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from hanja import hangul
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm

def mem_dict(df): 
    df = df.members   
    
    tmpmem = []
    tmppar = []
    hangle = []
    for i in df:
        for j in i:
            tmpmem.append(j[2])
            tmppar.append(j[1])
            hangle.append(j[0])
    
    mem_df = pd.DataFrame({'hanja':tmpmem,'hangle':hangle,'party':tmppar})
    mem_df = mem_df.drop_duplicates(subset = ['hanja','hangle'], keep = 'first').reset_index(drop = True)
    for idx,i in enumerate(mem_df.hangle):
        if len(i)>4:
            mem_df.drop(index = idx, inplace = True)
    
    mem_df.reset_index(drop = True, inplace = True)
    
    mem_dict = {}
    for idx, i in enumerate(mem_df.hanja):
        mem_dict[i] = [mem_df.hangle[idx],mem_df.party[idx]]
    
    return mem_dict 

def make_matrix(df):
    two_mode_matrix = []
    for i in tqdm(df.members):
        where = [x[2] for x in i]
        try:
            where = [idx_dict[x] for x in where]
        except:
            pass
        law_one= []
        for i in range(len(idx_dict)):
            if i in where:
                law_one.append(1.)
            else:
                law_one.append(0.)
        two_mode_matrix.append(law_one)
    
    two_mode_matrix = np.array(two_mode_matrix)
    one_mode_matrix = np.dot(two_mode_matrix.T,two_mode_matrix)

    return one_mode_matrix

def data_cleansing(search_name,matrix,delete = False):
    
    name = [name for name, hangle_party in all_dict.items() if hangle_party[0] == search_name]
    
    if delete == False:
        print(name)
        print('select remove name')
        remove_column = input()
        remove_column = int(remove_column)
        a = matrix.loc[name[0],]
        b = matrix.loc[name[1],]
        c = a+b
        matrix.loc[name[1],] = c
        
        matrix = matrix.drop(columns = name[remove_column], index = name[remove_column])
        all_name.remove(name[remove_column])
        all_dict.pop(name[remove_column],None)
    if delete == True:
        name = [name for name, hangle_party in all_dict.items() if hangle_party[0] == search_name][0]
        matrix = matrix.drop(columns = name, index = name)
        all_name.remove(name)
    return matrix

In [5]:
#%% 전처리

with open('data/raw2.pickle', 'rb') as f:
    raw2 = pickle.load(f)

all_dict = mem_dict(raw2)
all_name = sorted(list(set(all_dict)))

idx_dict = {} ## 벡터의 순서를 표현하기 위해 한자와 idx를 mapping
for idx, i in enumerate(all_name):
    idx_dict[i] = int(idx)


matrix = make_matrix(raw2) 
matrix = pd.DataFrame(matrix)
matrix.columns = all_name 
matrix.index = all_name 

100%|███████████████████████████████████████████████████████████████████████████| 2900/2900 [00:00<00:00, 18986.39it/s]


In [6]:
matrix

,丁慶姬,全惠淑,全海澈,全珠惠,兪京濬,兪義東,兪訂炷,具滋根,千俊鎬,南仁順,...,李鍾成,李鏞,李亮壽,李龍彬,李龍雨,林五卿,林昊宣,林炳憲,林鍾聲,林利子
丁慶姬,50.0,1.0,1.0,11.0,8.0,5.0,1.0,5.0,0.0,1.0,...,7.0,9.0,8.0,1.0,1.0,1.0,1.0,12.0,1.0,7.0
全惠淑,1.0,37.0,7.0,1.0,1.0,1.0,6.0,1.0,3.0,7.0,...,2.0,1.0,1.0,6.0,3.0,3.0,8.0,1.0,8.0,1.0
全海澈,1.0,7.0,95.0,1.0,1.0,1.0,7.0,1.0,3.0,7.0,...,1.0,1.0,1.0,12.0,7.0,4.0,10.0,1.0,20.0,1.0
全珠惠,11.0,1.0,1.0,32.0,8.0,4.0,1.0,7.0,0.0,1.0,...,3.0,9.0,2.0,1.0,1.0,1.0,1.0,5.0,1.0,6.0
兪京濬,8.0,1.0,1.0,8.0,129.0,9.0,1.0,30.0,0.0,1.0,...,21.0,21.0,8.0,1.0,1.0,1.0,1.0,23.0,1.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
林五卿,1.0,3.0,4.0,1.0,1.0,1.0,14.0,1.0,2.0,4.0,...,1.0,1.0,1.0,6.0,5.0,58.0,6.0,1.0,9.0,1.0
林昊宣,1.0,8.0,10.0,1.0,1.0,1.0,38.0,1.0,5.0,17.0,...,2.0,1.0,1.0,44.0,9.0,6.0,331.0,1.0,50.0,1.0
林炳憲,12.0,1.0,1.0,5.0,23.0,11.0,1.0,28.0,0.0,1.0,...,36.0,12.0,6.0,1.0,1.0,1.0,1.0,131.0,1.0,16.0
林鍾聲,1.0,8.0,20.0,1.0,1.0,3.0,31.0,2.0,3.0,8.0,...,1.0,1.0,1.0,24.0,6.0,9.0,50.0,1.0,170.0,1.0


In [7]:
'''
## 데이터 클렌징. 
## 오타로 인한 추가 구분 테이터를 통합. 

for i in matrix.columns:
    tmp = 0
    for j in i:
        tmp += hangul.is_hangul(j)
    if tmp != 0:
        print(i,all_dict[i])
        
matrix = data_cleansing('심상정',matrix)
matrix = data_cleansing('민병두',matrix)
    
## 의원직이 박탈된 사람은 제거.
with open(r'C:\Github\Project_Social_Bigdata_Assignment\Data\remove_list.pickle', 'rb') as f:
    remove_list = pickle.load(f)

for i in remove_list:
    try:
        matrix = data_cleansing(i,matrix, delete = True)
    except:
        print(i)

'''

In [9]:
# 전처리된 자료 저장.    
with open('data/matrix.pickle', 'wb') as f:
    pickle.dump(matrix , f, pickle.HIGHEST_PROTOCOL)

with open('data/all_dict.pickle', 'wb') as f:
    pickle.dump(all_dict , f, pickle.HIGHEST_PROTOCOL)